## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
#Use input statements to retrieve customer weather preferences, then use those preferences to identify potential 
#travel destinations and nearby hotels. Then, show those destinations on a marker layer map with pop-up markers.


# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config_weather import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df

,city_id,city,country,latitude,longitude,max_temp,humidity,cloudiness,wind_speed,weather_description
0,0,Ust-Ishim,RU,71.1665,57.6935,51.84,52,92,10.20,overcast clouds
1,1,San Patricio,US,-97.5169,28.0170,93.29,39,5,20.45,clear sky
2,2,Yellowknife,CA,-114.3525,62.4560,72.12,28,75,12.66,broken clouds
3,3,Kargil,IN,76.1000,34.5667,51.75,46,20,6.42,few clouds
4,4,Road Town,VG,-64.6167,18.4167,86.04,66,40,10.36,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
682,682,Gillette,US,-105.5022,44.2911,55.11,71,100,5.75,overcast clouds
683,683,Sorland,NO,12.6934,67.6670,44.46,62,99,17.94,overcast clouds
684,684,Lubao,PH,120.5500,14.9167,82.11,87,99,4.00,light rain
685,685,Novoagansk,RU,76.6625,61.9449,42.22,64,6,5.39,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like?"))

max_temp = float(input("What is the maximum temperature you would like?"))

type(min_temp) #string data type, need to convert it to float

What is the minimum temperature you would like?70
What is the maximum temperature you would like?90


float

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

temp_filter = city_data_df.loc[(city_data_df["max_temp"]>=min_temp) & (city_data_df["max_temp"]<=max_temp)]

temp_filter

,city_id,city,country,latitude,longitude,max_temp,humidity,cloudiness,wind_speed,weather_description
2,2,Yellowknife,CA,-114.3525,62.4560,72.12,28,75,12.66,broken clouds
4,4,Road Town,VG,-64.6167,18.4167,86.04,66,40,10.36,scattered clouds
5,5,Atuona,PF,-139.0333,-9.8000,78.10,69,27,14.85,scattered clouds
7,7,Rikitea,PF,-134.9692,-23.1203,72.79,57,0,15.28,clear sky
8,8,Vaini,TO,-175.2000,-21.2000,71.76,78,20,6.91,few clouds
...,...,...,...,...,...,...,...,...,...,...
678,678,Pandharpur,IN,75.3333,17.6667,75.60,73,97,16.15,overcast clouds
679,679,Haimen,CN,116.6142,23.1935,75.09,94,100,8.50,moderate rain
680,680,Talavera,PH,120.9333,15.6167,75.90,94,100,1.88,overcast clouds
684,684,Lubao,PH,120.5500,14.9167,82.11,87,99,4.00,light rain


In [5]:
# 4a. Determine if there are any empty rows.
temp_filter.count()


city_id                289
city                   289
country                289
latitude               289
longitude              289
max_temp               289
humidity               289
cloudiness             289
wind_speed             289
weather_description    289
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = temp_filter.dropna()

clean_df.count()

city_id                289
city                   289
country                289
latitude               289
longitude              289
max_temp               289
humidity               289
cloudiness             289
wind_speed             289
weather_description    289
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["city", "country", "max_temp", "weather_description", "latitude", "longitude"]].copy() 

# 5b. Create a new column "Hotel Name"
hotel_df["hotel_name"] = ""
hotel_df


,city,country,max_temp,weather_description,latitude,longitude,hotel_name
2,Yellowknife,CA,72.12,broken clouds,-114.3525,62.4560,
4,Road Town,VG,86.04,scattered clouds,-64.6167,18.4167,
5,Atuona,PF,78.10,scattered clouds,-139.0333,-9.8000,
7,Rikitea,PF,72.79,clear sky,-134.9692,-23.1203,
8,Vaini,TO,71.76,few clouds,-175.2000,-21.2000,
...,...,...,...,...,...,...,...
678,Pandharpur,IN,75.60,overcast clouds,75.3333,17.6667,
679,Haimen,CN,75.09,moderate rain,116.6142,23.1935,
680,Talavera,PH,75.90,overcast clouds,120.9333,15.6167,
684,Lubao,PH,82.11,light rain,120.5500,14.9167,


In [8]:
# 6a. Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["latitude"]
    lng=row["longitude"]
    params["location"]=f"{lng},{lat}" #accidentally switched lat and lon when making database
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "hotel_name"] = hotels["results"][0]["name"]
     
    except (IndexError):
        print("Hotel not found, skipping...")
        


Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...


In [13]:
hotel_df.head(10)


,city,country,max_temp,weather_description,latitude,longitude,hotel_name
624,Sabang,ID,82.56,light rain,95.3214,5.8933,Hotel Citra
627,Madhipura,IN,84.06,overcast clouds,86.7833,25.9167,SPOT ON 64083 Hotel Ganga
629,Sinor,IN,86.25,few clouds,73.3333,21.9167,Reva Forest Homestay
630,North Platte,US,73.29,overcast clouds,-100.7654,41.1239,"Holiday Inn Express & Suites North Platte, an ..."
634,Creel,MX,81.88,overcast clouds,-107.6333,27.7500,The Lodge at Creel Hotel & SPA
636,Half Moon Bay,US,85.59,mist,-122.4286,37.4636,"The Ritz-Carlton, Half Moon Bay"
639,Gizo,SB,81.55,light rain,156.8419,-8.1030,Imagination Island
640,Bac Lieu,VN,79.97,overcast clouds,105.7244,9.2850,Khách sạn Sài Gòn Bạc Liêu
642,Samarinda,ID,72.34,overcast clouds,117.1500,-0.5000,ASTON Samarinda Hotel & Convention Center
643,Metro,ID,74.10,broken clouds,105.3067,-5.1131,OYO 3177 Wisma Lintang


In [44]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

clean_hotel_df

,city,country,max_temp,weather_description,latitude,longitude,hotel_name
4,Road Town,VG,86.04,scattered clouds,-64.6167,18.4167,"ZINGARA 76ft Catamaran, full crew-all included"
14,Kahului,US,82.42,scattered clouds,-156.4700,20.8947,Maui Seaside Hotel
34,Butaritari,KI,80.83,light rain,172.7902,3.0707,Isles Sunset Lodge
39,Kapaa,US,80.11,scattered clouds,-159.3190,22.0752,Sheraton Kauai Coconut Beach Resort
46,Georgetown,MY,82.33,few clouds,100.3354,5.4112,Cititel Penang
...,...,...,...,...,...,...,...
662,Mancio Lima,BR,86.07,broken clouds,-72.8958,-7.6142,Motel Asas Do Desejo
665,Semnan,IR,81.99,overcast clouds,53.3971,35.5729,Semnan Tourist Hotel
675,Sakassou,CI,80.44,overcast clouds,-5.2926,7.4546,Parents de Damienne
684,Lubao,PH,82.11,light rain,120.5500,14.9167,Hacienda Gracia


In [48]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [46]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
<dt>Weather Description</dt><dd>{weather_description}</dd>
<dt>Max Temp</dt><dd>{max_temp}</dd>
<dt>Hotel Name</dt><dd>{hotel_name}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
location = clean_hotel_df[["longitude", "latitude"]]

In [47]:
fig=gmaps.figure(center=(31.0,31.0), zoom_level=1.5)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(location, info_box_content=hotel_info)

# 11b. Display the figure
fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))